# LSTM Model Type Extra

In [1]:
import pandas as pd
import numpy as np
from numpy import array
from collections import defaultdict

import keras_tuner
from keras.layers import LSTM
from keras.layers import Dense
from keras.models import Sequential
import tensorflow as tf

from LSTM_Model_Type_Extra_Scale import split_position, LSTM_FitPredict

### Set Random Seed for Code Reproductability

In [2]:
import random
random.seed(2020)

from numpy.random import seed
seed(2020)
tf.random.set_seed(2020)

import random as python_random
python_random.seed(2020)

### Read in Raw Data & Preprocessing

In [3]:
# Read in the raw age-sex cohort data without the remainder area
# Prepare the unique sa3 names, codes for LSTM fitting selection
raw_data = pd.read_csv('../Data/true_1000_fulldata.csv')
sa3_num = len(raw_data['SA3 Code'].unique())
sa3_names = raw_data['SA3 Name'].unique()
sa3_codes = raw_data['SA3 Code'].unique()

In [4]:
# Key Year for Model Fitting
train_start = 1991
train_end = 2001

### Get minimum and maximum across all cohorts

In [5]:
training_min = raw_data.loc[raw_data["Year"] <= train_end, ['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']].min().min()
training_max = raw_data.loc[raw_data["Year"] <= train_end, ['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']].max().max()

### Rescale data points

In [6]:
data_range = training_max-training_min
    
rescaled_data = raw_data.copy()
rescaled_data.loc[:,['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']] = rescaled_data.loc[:,['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']] - training_min
rescaled_data.loc[:,['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']] = rescaled_data.loc[:,['m0-4','m5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+','f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']] * (1/data_range)

### Preprocessing by Splitting Population into Sex-Group

In [7]:
# Split the data by sex
age_groups = ['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74','75-79', '80-84', '85+']
population_m_dict = defaultdict(dict)
population_f_dict = defaultdict(dict)

# save population for each year into lists
for sa3_code in sa3_codes:
  population_m_dict[sa3_code] = dict()
  population_f_dict[sa3_code] = dict()
  for year in range(1991,2012):
    if(rescaled_data[(rescaled_data['Year']==year) & (rescaled_data['SA3 Code']==sa3_code)]['Total'].size>0):
      population_m_dict[sa3_code][year] = rescaled_data[(rescaled_data['Year']==year) & (rescaled_data['SA3 Code']==sa3_code)][['m0-4',
                              'm5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+']].values.tolist()[0]
      population_f_dict[sa3_code][year] = rescaled_data[(rescaled_data['Year']==year) & (rescaled_data['SA3 Code']==sa3_code)][['f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']].values.tolist()[0]

### Create Dataframe for Storing Prediction Result

In [8]:
output = pd.DataFrame(index = range(sa3_num*36), columns = ['Code','Area name','Sex','Age group',2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011])
for sa3_code in range(0,sa3_num):
  for age_group in range(18):
    output.loc[sa3_code*36+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Females','Age group':age_groups[age_group]}
    output.loc[sa3_code*36+18+age_group] = {'Code':sa3_codes[sa3_code],'Area name':sa3_names[sa3_code],'Sex':'Males','Age group':age_groups[age_group]}

### Start LSTM Fitting & Predicting

##### Define Variable for Splitting the Full Dataset

In [9]:
# choose a number of time steps
gap = 5

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 18
n_age_groups = 18

# Number of Epoch(es)
epoch = 1000

# Obtain the split position for the training sex, validation sex, and the first input_x
train_val_bounds, test_bounds = split_position(gap, train_start, train_end)

##### LSTM Model Defining

In [10]:
'''Function for Defining the LSTM Model'''
def lstm_model(lstm1_units, activ, optimizer, loss_fun):
    model = Sequential()
    model.add(LSTM(lstm1_units, activation=activ, input_shape=(1, n_features)))
    model.add(Dense(n_age_groups))
    model.compile(optimizer=optimizer, loss=loss_fun, metrics=["mean_absolute_percentage_error"])
    return model

##### LSTM Model Tunning

In [11]:
# Define HYPERPARAMETERS
lstm1_min = 100
lstm1_max = 1000
lstm1_step = 100
activ_functions = ["relu"]
optimizers = ["adam","adagrad"]
loss_functions = ["mse"]

'''In-Build Function for LSTM Model Tuning'''
def build_model(hp):
    lstm1_units = hp.Int("LSTM units", min_value=lstm1_min,max_value=lstm1_max,step=lstm1_step)
    activation = hp.Choice("Activation Function", activ_functions)
    optimizer = hp.Choice("Optimizer", optimizers)
    loss_fun = hp.Choice("Loss Function", loss_functions)
    model = lstm_model(lstm1_units, activation, optimizer, loss_fun)
    return model

# Generate the tuned model with keras_tuner package
tuner = keras_tuner.RandomSearch(hypermodel=build_model, objective=keras_tuner.Objective("val_mean_absolute_percentage_error", "min"))

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2022-09-13 00:14:40.687371: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-09-13 00:14:40.687747: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


##### Generate Prediction Result

In [12]:
# Generate each sex's Prediction Result of each age-cohort for all areas and store them into csv
output = LSTM_FitPredict(sa3_codes, population_m_dict, gap, train_val_bounds, test_bounds, n_features, epoch, "Males", tuner, output, training_min, training_max)
output = LSTM_FitPredict(sa3_codes, population_f_dict, gap, train_val_bounds, test_bounds, n_features, epoch, "Females", tuner, output, training_min, training_max)
output.to_csv('../Data/Projection/Iterative_predict_output_Step1_1000_Type_Extra_Scale.csv',index=False,header=True)

2022-09-13 00:14:41.215378: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-09-13 00:14:41.548399: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 00:14:41.954548: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-13 00:15:25.065855: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 00:15:25.397444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-13 00:16:36.637512: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 00:16:36.963374: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-13 00:17:38.106444: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 00:17:38.444611: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2022-09-13 00:18:45.743942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 00:18:46.060754: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

2006
[[0.2261012  0.3198939  0.3464947  0.35626972 0.23126058 0.21718837
  0.26429743 0.33715546 0.3701863  0.3773814  0.36441782 0.19755389
  0.12694514 0.083947   0.07114772 0.06011152 0.03884089 0.02163279]]
[[2450.937   3467.65    3756.0027  3861.9636  2506.8647  2354.322
  2864.9841  3654.7651  4012.8196  4090.8145  3950.289   2141.4841
  1376.0853   909.9854   771.2413   651.6089   421.03525  234.49945]]
2011
[[0.13455398 0.2560087  0.30606917 0.3429262  0.19123231 0.13560915
  0.18562856 0.2702973  0.33200476 0.36738712 0.40621096 0.22391537
  0.14584056 0.08836161 0.08188692 0.07267584 0.0516384  0.02980545]]
[[1458.5652  2775.1345  3317.7898  3717.32    2072.9583  1470.0032
  2012.2136  2930.0227  3598.9316  3982.4763  4403.3267  2427.2427
  1580.9116   957.8398   887.6542   787.8061   559.76025  323.09106]]
INFO:tensorflow:Oracle triggered exit


2022-09-13 06:22:31.417473: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 06:22:32.589769: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 06:23:05.481965: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2006
[[0.05302437 0.06293546 0.06684802 0.06840199 0.06210286 0.07791714
  0.07337959 0.07388727 0.08034516 0.07995667 0.12485968 0.08959384
  0.05535724 0.03814319 0.03365642 0.02921733 0.02500697 0.0211664 ]]
[[ 574.7842   682.22034  724.63257  741.4776   673.195    844.62177
   795.43475  800.93805  870.9415   866.7303  1353.4789   971.19727
   600.0725   413.47217  364.83557  316.7158   271.0756   229.44377]]
2011
[[0.05209226 0.06250733 0.0662944  0.06752642 0.06010743 0.07736619
  0.07344964 0.07320625 0.0799053  0.07738829 0.12576689 0.09111784
  0.05600255 0.03831986 0.0339096  0.02963378 0.02546699 0.0219254 ]]
[[ 564.68005  677.57947  718.6313   731.9864   651.5645   838.6495
   796.1941   793.5557   866.17346  838.8891  1363.3131   987.7174
   607.0676   415.38733  367.5801   321.23016  276.06213  237.67134]]
INFO:tensorflow:Oracle triggered exit


2022-09-13 06:23:07.442200: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 06:23:08.800783: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-09-13 06:23:41.903311: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


2006
[[0.07454476 0.08911745 0.09061887 0.08953346 0.09163246 0.09629636
  0.09855008 0.11457406 0.11881229 0.10820954 0.10753214 0.11323961
  0.09371049 0.0698204  0.05776805 0.05928735 0.03619938 0.03272557]]
[[ 808.0652   966.0332   982.3086   970.54266  993.29584 1043.8525
  1068.2828  1241.9828  1287.9252  1172.9915  1165.6484  1227.5173
  1015.8217   756.85315  626.2057   642.6749   392.40125  354.7452 ]]
2011
[[0.0741158  0.09054506 0.09141807 0.08902849 0.08800999 0.09219389
  0.0976033  0.115588   0.12182305 0.10805943 0.10561386 0.1160785
  0.09805788 0.07157977 0.05863261 0.06394709 0.03879231 0.0354313 ]]
[[ 803.4152   981.5084   990.9718   965.0688   954.0283   999.3817
  1058.0198  1252.974   1320.5619  1171.3643  1144.8542  1258.2909
  1062.9474   775.9247   635.57745  693.18646  420.50864  384.07532]]
